In [ ]:
import uuid

import astrocast.sim as sim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import importlib
from IPython.display import clear_output
import logging
import time
from pathlib import Path
import shutil
import humanize

In [ ]:
importlib.reload(sim)
# logging.basicConfig(level=logging.INFO)

env_dict = dict(molecules=["glutamate", "repellent", "calcium"], diffusion_rate=0.1,
                dt=25,  # 1
                degradation_factor=0.1, dtype='float16')

grm_dict = dict(num_branches=32, z_thickness=3, jitter=2, release_amplitude=50,
                stochastic_probability=0.1,
                # signal_function=lambda x: 0 if x % 5 != 0 else 0.4,
                signal_function=lambda x: 0,
                )

ast_dict = dict(radius=3, num_branches=4, max_branch_radius=1, min_radius=0.001,
                start_spawn_radius=0.9, spawn_length=3, spawn_radius_factor=0.5,
                growth_factor=0.05, max_tries=5,
                allow_pruning=True,
                atp_cost_per_glutamate=-180, atp_cost_per_unit_surface=1,
                molecules=dict(glutamate=0, calcium=0, ATP=5),
                min_steepness=0.005, min_trend_amplitude=0.01,
                repellent_volume_factor=0, repellent_surface_factor=0,
                repellent_concentration=0  # 1
                )

my_sim = sim.Simulation(num_astrocytes=7, grid_size=(100, 100), border=10,
                        environment_dict=env_dict, glutamate_release_param=grm_dict,
                        astrocyte_param=ast_dict)
my_sim.plot()

img_dir = Path("imgs/")
if img_dir.exists():
    shutil.rmtree(img_dir)
    img_dir.mkdir()
else:
    img_dir.mkdir()

my_sim.data_logger.save(Path("log.txt"))

step_counter = 1

In [ ]:
for epoch in range(32):
    
    clear_output(wait=True)
    
    # plotting
    my_sim.plot(last_n_messages=16)
    display(my_sim.fig)
    my_sim.fig.savefig(img_dir.joinpath(f"img_{step_counter}.png"), dpi=(80))
    
    # for step in range(max(2, int(np.log2(step_counter)))):
    t0 = time.time()
    for step in range(1):
        
        my_sim.run_simulation_step()
        step_counter += 1
        
        # export log
        my_sim.data_logger.save(Path("log.txt"))
    
    print(f"epoch runtime: {humanize.naturaldelta(time.time() - t0)}")


In [ ]:
# my_sim.plot(last_n_messages=15)
# display(my_sim.fig)
# my_sim.fig.savefig(img_dir.joinpath(f"img_{step_counter}.png"), dpi=(80))
# 
# my_sim.data_logger.save(Path("log.txt"))

In [ ]:
my_sim.astrocytes[0].plot_branch_history()


In [ ]:
# branches = my_sim.astrocytes[0].branches
# 
# thickness = []
# for branch in branches:
#     avg_radius = np.mean(branch.start.radius + branch.end.radius)
#     thickness.append(avg_radius)
# 
# print(f"{np.mean(thickness):.2f} +- {np.std(thickness):.2f}")

In [ ]:
# fig, ax = plt.subplots(1, 1)
# ax.set_xscale('log')
# ax.set_yscale('log')
# _ = ax.hist(thickness, bins=1000)

In [ ]:
branch0 = None
branch_id = "92ede384"

branches = []
for ast in my_sim.astrocytes:
    branches += ast.branches

for branch in branches:
    if branch.get_short_id() == branch_id:
        branch0 = branch
        break

print(branch0)

if branch0 is not None:
    
    trend = branch0.get_trend("glutamate", intra=False)
    print(f"{trend:.1E}, {trend / ast_dict['min_trend_amplitude'] * 100:.1f}%")
    
    fig, ax = plt.subplots()
    
    hist = branch0.extracellular_history["glutamate"]
    ax.plot(hist)
    